## 04- Explicabilidad y Despliegue de la Solución

En este notebook evaluaremos una parte importante del desarrollo de herramientas analíticas como lo es la explicabilidad del mejor modelo hallado en el notebook anterior. A su vez, introduciremos el procedimiento para desplegarlo a través de un docker container.

In [1]:
## cargar modelo anterior
from model_parameters import *
import pickle
import pandas as pd
import numpy as np
from sklearn import set_config
set_config(display='diagram')

with open("models/xgbmodel.pickle", 'rb') as f:
    xgb = pickle.load(f)
xgb

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('cleaner',
                                        ColumnTransformer(sparse_threshold=0,
                                                          transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value=0,
                                                                                                        strategy='constant')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['n_prod_prev',
                                                                          'total_siniestros',
                                                                          'total_pagado_smmlv',
                                                                          'anios_ultimo_siniestro',
                                                                          'Activos__c',
                                                                          'AnnualRevenue',
                                                                          'MontoAnual__c',
                                                                          'Otros...
                                                                          'TipoVehiculo__c',
                                                                          'EstadoCivil__pc',
                                                                          'Genero__pc',
                                                                          'ciudad_name'])])),
                                       ('pca', PCA()),
                                       ('model', GradientBoostingClassifier())],
                                verbose=True),
             n_jobs=-1,
             param_grid={'model__learning_rate': [0.01],
                         'model__max_depth': [5, 10, 20],
                         'model__min_samples_split': [2, 5],
                         'model__n_estimators': [50, 100, 250],
                         'model__subsample': [0.9, 1],
                         'pca__n_components': [10, 20]},
             scoring='f1', verbose=True)

In [2]:
# cargar data de test
train=pd.read_csv("train.csv",sep=";",dtype={'Asegurado__c':str,'NumeroPoliza__c':str,'CodigoTipoAsegurado__c':str,'churn':int,'ClaseVehiculo__c':str,'TipoVehiculo__c':str,'PuntoVenta__c':str,'MarcaVehiculo__c':str}).drop(columns='Asegurado__c')
test=pd.read_csv("test.csv",sep=";",dtype={'Asegurado__c':str,'NumeroPoliza__c':str,'CodigoTipoAsegurado__c':str,'churn':int,'ClaseVehiculo__c':str,'TipoVehiculo__c':str,'PuntoVenta__c':str,'MarcaVehiculo__c':str}).drop(columns='Asegurado__c')


## Shap Values


In [3]:
import shap
explainer = shap.Explainer(xgb.best_estimator_[2])

In [4]:
data_exp=xgb.best_estimator_[0:2].transform(train)

In [5]:
shap_values = explainer(data_exp)

# visualize the first prediction's explanation
#shap.plots.waterfall(shap_values[0])

## Despliegue de la solución

Una de las formas más comunes de disponibilizar las soluciones de analítica avanzada es a través de un servicio web. Con Fast API este proceso se facilita. Una posible app del servicio de detección de churn se muestra a continuación.